In [1]:
import pandas as pd
import numpy as np

import os

import featuretools as ft

import utils

import warnings
warnings.filterwarnings('ignore')

In [2]:
data_dir = '../data/'
filenames = os.listdir(data_dir)
for i, ele in enumerate(filenames):
    print(i, ele)

0 olist_customers_dataset.csv
1 olist_geolocation_dataset.csv
2 olist_orders_dataset.csv
3 olist_order_items_dataset.csv
4 olist_order_payments_dataset.csv
5 olist_order_reviews_dataset.csv
6 olist_products_dataset.csv
7 olist_sellers_dataset.csv
8 processed
9 product_category_name_translation.csv


## Reviews data

In [3]:
df_reviews = pd.read_csv(data_dir+filenames[5], parse_dates=['review_creation_date', 
                                                             'review_answer_timestamp'])



In [4]:
df_reviews['review_id'].nunique(),df_reviews['order_id'].nunique()

(99173, 99441)

In [5]:
df_reviews.isna().sum()/df_reviews.shape[0]*100

review_id                   0.000
order_id                    0.000
review_score                0.000
review_comment_title       88.285
review_comment_message     58.247
review_creation_date        0.000
review_answer_timestamp     0.000
dtype: float64

## create features
* length of review title
* length of review message
* words of review message
* words of review title
* difference between delivery date and review date

In [6]:
df_reviews.fillna('', inplace=True)

df_reviews['review_comment_length'] = df_reviews['review_comment_message'].apply(len)
df_reviews['review_comment_words'] = df_reviews['review_comment_message'].apply(lambda x:len(x.split()))

df_reviews['review_title_length'] = df_reviews['review_comment_title'].apply(len)
df_reviews['review_title_words'] = df_reviews['review_comment_title'].apply(lambda x:len(x.split()))

## Map reviews with orders dataset

In [7]:
df_orders = utils.read_pickles('../data/processed/orders')

100%|██████████| 3/3 [00:00<00:00, 12.17it/s]


## Considering only delivered orders without any missing values

In [8]:
df_orders = df_orders[df_orders['order_status']==3]
# df_orders.isna().sum()
df_orders.dropna(inplace=True)

## Merge both datasets and build features
* Difference in review given and delivery date
* Difference in estimated and delivery date
* Order month and year

In [9]:
df_orders = df_orders.merge(df_reviews[['order_id', 'review_score', 'review_creation_date', 
                            'review_comment_length', 'review_comment_words']], on='order_id')

In [10]:
df_orders['review_delivered_gap'] = (df_orders['review_creation_date'] - \
                        df_orders['order_delivered_customer_date'].dt.date.astype('datetime64[D]')).dt.days


df_orders['estimated_delivery_gap'] =(df_orders['order_delivered_customer_date'].dt.date.\
                                      astype('datetime64[D]') -\
                                      df_orders['order_estimated_delivery_date'].dt.date.\
                                      astype('datetime64[D]')).dt.days

In [11]:
df_orders['order_year'] = df_orders['order_purchase_timestamp'].dt.year
df_orders['order_month'] = df_orders['order_purchase_timestamp'].dt.month


### Note: order_id is unique to every customer, clustering for customers can be similar to clustering for orders

In [12]:
df_orders['order_id'].nunique(), df_orders['customer_id'].nunique()

(96455, 96455)

## Dropping date columns

In [13]:
df_orders.drop(['customer_id', 'order_purchase_timestamp','order_approved_at',
                'order_delivered_carrier_date','order_delivered_customer_date',
                'order_estimated_delivery_date','review_creation_date'], axis=1,inplace=True)


## Items dataset

In [14]:
df_items = pd.read_csv(data_dir+filenames[3], parse_dates=['shipping_limit_date'])

## Creating order level features 

In [15]:
aggs = {'order_item_id':['count', 'sum'],
       'product_id':'nunique',
       'seller_id':'nunique',
       #'shipping_limit_date': ['min', 'max'],  ## not used
       'price':['sum', 'mean', 'min', 'max'],
       'freight_value':['sum', 'mean', 'min', 'max']}

df_items = df_items.groupby('order_id').agg(aggs).reset_index()

In [16]:
df_items.columns = ['order_id', 'order_items_counts', 'order_item_sum', 'product_id_count',
                    'seller_id_count', #'shipping_min', 'shipping_max', 
                   'price_sum','price_mean','price_min','price_max',
                   'freight_sum','freight_mean','freight_min','freight_max']

In [17]:
## merge with order level information
df_orders = df_orders.merge(df_items, on='order_id')

## Payments data

In [18]:
df_payments = utils.read_pickles('../data/processed/payments')

100%|██████████| 3/3 [00:00<00:00, 38.11it/s]


## creating order level features

In [19]:
aggs = { 'payment_sequential' :['count', 'sum'],
        'payment_type':['sum'],
        'payment_installments': ['sum'],
        'payment_value':['sum','max']    
}

df_payments = df_payments.groupby('order_id').agg(aggs).reset_index()
df_payments.columns = ['order_id', 'payment_sequential_count','payment_sequential_sum', 'payment_type_sum',
                       'payment_installments_sum', 'payment_value_sum', 'payment_value_max']


In [20]:
## merge with order data
df_orders = df_orders.merge(df_payments, on='order_id')

## Product and seller level information not picked for now

In [21]:
# df_products = utils.read_pickles('../data/processed/products')
# df_sellers = utils.read_pickles('../data/processed/sellers')

In [22]:
df_orders.dtypes

order_id                     object
order_status                   int8
review_score                  int64
review_comment_length         int64
review_comment_words          int64
review_delivered_gap          int64
estimated_delivery_gap        int64
order_year                    int64
order_month                   int64
order_items_counts            int64
order_item_sum                int64
product_id_count              int64
seller_id_count               int64
price_sum                   float64
price_mean                  float64
price_min                   float64
price_max                   float64
freight_sum                 float64
freight_mean                float64
freight_min                 float64
freight_max                 float64
payment_sequential_count      int64
payment_sequential_sum      float64
payment_type_sum               int8
payment_installments_sum       int8
payment_value_sum           float16
payment_value_max           float16
dtype: object

In [23]:
utils.to_pickles(df_orders, '../data/processed/cluster_features')

3it [00:00,  8.42it/s]


In [ ]:
# Create entityset
entity_set = ft.EntitySet(id = 'order_val')

# order entity
entity_set = entity_set.entity_from_dataframe(
    entity_id='orders',
    dataframe=df_orders,
    index='order_id',
    variable_types = {'order_status': ft.variable_types.Categorical}
)

# item entity
entity_set = entity_set.entity_from_dataframe(
    entity_id='items',
    dataframe=df_items,
    make_index=True,
    index = 'item_id',
    time_index = 'shipping_limit_date'
)

# payments entity
entity_set = entity_set.entity_from_dataframe(
    entity_id='payments',
    dataframe=df_payments,
    make_index=True,
    index='payment_id',
    variable_types = {'payment_type': ft.variable_types.Categorical}
)

# customers entity
entity_set = entity_set.entity_from_dataframe(
    entity_id='customers',
    dataframe=df_customers,
    index='customer_id',
    variable_types = {'customer_city': ft.variable_types.Categorical,
                     'customer_state': ft.variable_types.Categorical}
)

# product entity
entity_set = entity_set.entity_from_dataframe(
    entity_id='products',
    dataframe=df_products,
    index='product_id',
    variable_types = {'product_category_name': ft.variable_types.Categorical}
)

# seller entity
entity_set = entity_set.entity_from_dataframe(
    entity_id='sellers',
    dataframe=df_sellers,
    index='seller_id',
    variable_types = {'seller_city': ft.variable_types.Categorical,
                     'seller_state': ft.variable_types.Categorical}
)